In [1]:
import os
import sys
# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))

In [2]:
import yfinance as yf 
import pandas as pd 
import numpy as np
from datetime import datetime  
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [3]:
end=datetime.now()
start=datetime(end.year-15,end.month,end.day)
stock=('BTC-USD')
stock_data=yf.download(stock,start=start,end=end)

[*********************100%***********************]  1 of 1 completed


In [4]:
#closing price history
closing_price=stock_data[['Close']]

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(closing_price[['Close']].dropna())

In [7]:
len(scaled_data)

3773

In [8]:
#prepare data for LSTM  
x_data=[]
y_data =[]
base_days=100
for i in range(base_days,len(scaled_data)):
    x_data.append(scaled_data[i-base_days: i])
    y_data. append(scaled_data[i])
x_data= np.array(x_data)
y_data= np.array(y_data)

In [9]:
#split data into training and testing
train_size= int(len(x_data)*0.9)
x_train,y_train=x_data[:train_size],y_data[:train_size]
x_test,y_test=x_data[train_size:],y_data[train_size:] 

In [10]:
#reshape data for LSTM
model=Sequential(
    [
        LSTM(128,return_sequences=True,input_shape=(x_train.shape[1],1)),
        LSTM(64,return_sequences=False),
        Dense(25),
        Dense(1)
    ]
)
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

d:\My project\Cryptocurrency-Bitcoin-Price-Prediction\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 128)       │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)